In [ ]:
%%sh
pip install Pygments -q
echo "Please restart your runtime now (Runtime -> Restart runtime). This ensures that the correct dependencies are loaded."

# https://github.com/feast-dev/feast-gcp-fraud-tutorial/blob/main/notebooks/Validating_Online_Features_While_Detecting_Fraud.ipynb

In [1]:
!rm -rf dqm_repo
!feast init dqm_repo


Creating a new Feast repository in /home/francisco/github/feast/examples/data-quality-monitoring/dqm_repo.



In [2]:
%cd dqm_repo
!mkdir logs
!ls -R

/home/francisco/github/feast/examples/data-quality-monitoring/dqm_repo
.:
data  example.py  feature_store.yaml  __init__.py  logs

./data:
driver_stats.parquet

./logs:


In [3]:
!pygmentize feature_store.yaml

project: dqm_repo
registry: data/registry.db
provider: local
online_store:
    path: data/online_store.db
entity_key_serialization_version: 2


# Taking a look at our data

In [4]:
import pandas as pd

pd.read_parquet("data/driver_stats.parquet").head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2022-07-29 06:00:00+00:00,1005,0.617167,0.021683,433,2022-08-13 06:14:26.993
1,2022-07-29 07:00:00+00:00,1005,0.814462,0.942930,446,2022-08-13 06:14:26.993
2,2022-07-29 08:00:00+00:00,1005,0.749975,0.270762,467,2022-08-13 06:14:26.993
3,2022-07-29 09:00:00+00:00,1005,0.858458,0.396312,500,2022-08-13 06:14:26.993
4,2022-07-29 10:00:00+00:00,1005,0.386625,0.168782,699,2022-08-13 06:14:26.993


In [5]:
!pygmentize -f terminal16m example.py

# This is an example feature definition file

from datetime import timedelta

from feast import Entity, FeatureService, FeatureView, Field, FileSource
from feast.types import Float32, Int64

# Read data from parquet files. Parquet is convenient for local development mode. For
# production, you can use your favorite DWH, such as BigQuery. See Feast documentation
# for more info.
driver_hourly_stats = FileSource(
    name="driver_hourly_stats_source",
    path="/home/francisco/github/feast/examples/data-quality-monitoring/dqm_repo/data/driver_stats.parquet",
    timestamp_field="event_timestamp",
    created_timestamp_column="created",
)

# Define an entity for the driver. You can think of entity as a primary key used to
# fetch features.
driver = Entity(name="driver", join_keys=["driver_id"])

# Our parquet files contain sample data that includes a driver_id column, timestamps and
# three feature column. Here we define a Feature View that will allow us to serve this
# data to our model 

# Configurations for online evaluation

In [6]:
feature_store = \
f"""project: dqm_repo
registry: data/registry.db
provider: local
offline_store:
    type: file
online_store:
    path: data/online_store.db
entity_key_serialization_version: 2
feature_server:
    enabled: True
    feature_logging:
        enabled: True
        flush_interval_secs: 60
        write_to_disk_interval_secs: 10
    
go_feature_retrieval: False
"""

with open('feature_store.yaml', "w") as feature_store_file:
    feature_store_file.write(feature_store)

dqm_services = f"""
from feast import FeatureService
from feast.feature_logging import LoggingConfig
from feast.infra.offline_stores.file_source import FileLoggingDestination

from example import driver_hourly_stats, driver, driver_hourly_stats_view

fs = FeatureService(
    name="driver_activity", 
    features=[driver_hourly_stats_view],
    logging_config=LoggingConfig(
        sample_rate=1.0,
        destination=FileLoggingDestination(path="logs/"),
    )
)
"""
    
with open('dqm_services.py', "w") as dqm_services_file:
    dqm_services_file.write(dqm_services)

In [7]:
!feast apply

Created entity driver
Created feature view driver_hourly_stats
Created feature service driver_activity

Created sqlite table dqm_repo_driver_hourly_stats



In [8]:
from datetime import datetime, timedelta
import pandas as pd

from feast import FeatureStore
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage

# The entity dataframe is the dataframe we want to enrich with feature values
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003],
        "label_driver_reported_satisfaction": [1, 5, 3], 
        "event_timestamp": [
            datetime.now() - timedelta(minutes=11),
            datetime.now() - timedelta(minutes=36),
            datetime.now() - timedelta(minutes=73),
        ],
    }
)

store = FeatureStore(repo_path=".")

training_data_job = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
)

reference_dataset = store.create_saved_dataset(
    from_=training_data_job,
    name="reference_dataset",
    storage=SavedDatasetFileStorage(path='data/driver_stats_reference.parquet')
)

print(reference_dataset)

{
  "spec": {
    "name": "reference_dataset",
    "features": [
      "driver_hourly_stats:conv_rate",
      "driver_hourly_stats:acc_rate",
      "driver_hourly_stats:avg_daily_trips"
    ],
    "joinKeys": [
      "driver_id",
      "label_driver_reported_satisfaction"
    ],
    "storage": {
      "fileStorage": {
        "fileFormat": {
          "parquetFormat": {}
        },
        "uri": "data/driver_stats_reference.parquet"
      }
    }
  },
  "meta": {
    "createdTimestamp": "2022-08-13T12:14:57.925228Z",
    "minEventTimestamp": "2022-08-13T05:01:57.842691Z",
    "maxEventTimestamp": "2022-08-13T06:03:57.842684Z"
  }
}


/home/francisco/github/feast/sdk/python/feast/feature_store.py:1127: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


# Creating our own Expectations Profiler

In [52]:
from feast.dqm.profilers.ge_profiler import ge_profiler
from great_expectations.dataset import PandasDataset
from great_expectations.core.expectation_suite import ExpectationSuite

# this profiler should pass
@ge_profiler
def user_features_profiler(dataset: PandasDataset) -> ExpectationSuite:
    print(dataset.columns)
    dataset.expect_column_to_exist("driver_id")
    dataset.expect_column_values_to_be_between("avg_daily_trips", 0, 1000)
    dataset.expect_column_values_to_be_between("conv_rate", 0, 1)
    dataset.expect_column_values_to_be_between("acc_rate", 0, 1)
    return dataset.get_expectation_suite()

# this profiler should trigger failures
@ge_profiler
def user_features_profiler_fail(dataset: PandasDataset) -> ExpectationSuite:
    dataset.expect_column_to_exist("this should fail")
    # dataset.expect_column_values_to_be_between("avg_daily_trips", )
    return dataset.get_expectation_suite()

ds = store.get_saved_dataset('reference_dataset')
validation_reference = ds.as_reference(name='user_features_profiler', profiler=user_features_profiler)
validation_reference_fail = ds.as_reference(name='user_features_profiler_fail', profiler=user_features_profiler_fail)

# print(training_data_job.to_df(validation_reference=validation_reference))
# print(training_data_job.to_df(validation_reference=validation_reference_fail))

/home/francisco/github/feast/sdk/python/feast/feature_store.py:1180: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [73]:
@ge_profiler
def profiler_with_unrealistic_expectations(dataset: PandasDataset) -> ExpectationSuite:
    # need to create dataframe with corrupted data first
    df = pd.DataFrame()
    df["current_balance"] = [100]
    df["avg_passenger_count"] = [0]

    other_ds = PandasDataset(df)
    other_ds.expect_column_max_to_be_between("current_balance", -1000, -100)
    other_ds.expect_column_values_to_be_in_set("avg_passenger_count", value_set={0})

    # this should pass
    other_ds.expect_column_min_to_be_between("avg_passenger_count", 0, 1000)

    return other_ds.get_expectation_suite()

@ge_profiler
def profiler_with_unrealistic_expectations2(dataset: PandasDataset) -> ExpectationSuite:
    # need to create dataframe with corrupted data first
    df = pd.DataFrame()
    df["current_balance"] = [-100]
    df["avg_passenger_count"] = [0]

    other_ds = PandasDataset(df)
    other_ds.expect_column_max_to_be_between("current_balance", -1000, -100)
    other_ds.expect_column_values_to_be_in_set("avg_passenger_count", value_set={0})

    # this should pass
    other_ds.expect_column_min_to_be_between("avg_passenger_count", 0, 1000)

    return other_ds.get_expectation_suite()


In [79]:
ds.get_profile(profiler=profiler_with_unrealistic_expectations) ==  ds.get_profile(profiler=profiler_with_unrealistic_expectations2) 

False

In [53]:
ds.get_profile(profiler=user_features_profiler_fail)

<GEProfile with expectations: []>

In [54]:
type(user_features_profiler), type(ds), type(ds.to_df())

(feast.dqm.profilers.ge_profiler.GEProfiler,
 feast.saved_dataset.SavedDataset,
 pandas.core.frame.DataFrame)

In [62]:
from great_expectations.dataset import PandasDataset
from feast.dqm.profilers.ge_profiler import _prepare_dataset, _add_feature_metadata, GEProfile
import great_expectations as ge

In [56]:
dataset = _prepare_dataset(PandasDataset(ds.to_df()))
# this doesn't appear to do anything but it's in the `analyze_dataset` method
dataset = _add_feature_metadata(dataset)
# It looks like when it fails it doesn't return anything
gething = user_features_profiler_fail.user_defined_profiler(dataset)

In [57]:
gething

{
  "data_asset_type": "Dataset",
  "expectation_suite_name": "default",
  "expectations": [],
  "meta": {
    "great_expectations_version": "0.14.13"
  },
  "ge_cloud_id": null
}

In [64]:
# and now we reproduce the empty expectation
GEProfile(expectation_suite=gething)

<GEProfile with expectations: []>

In [65]:
# so the root is the profiler and gething is returning nothing

In [66]:
user_features_profiler_fail.user_defined_profiler(dataset)

{
  "data_asset_type": "Dataset",
  "expectation_suite_name": "default",
  "expectations": [],
  "meta": {
    "great_expectations_version": "0.14.13"
  },
  "ge_cloud_id": null
}

In [67]:
type(user_features_profiler_fail), 

feast.dqm.profilers.ge_profiler.GEProfiler

In [49]:
ge.validate(dataset, expectation_suite=gething, result_format="COMPLETE")

{
  "statistics": {
    "evaluated_expectations": 0,
    "successful_expectations": 0,
    "unsuccessful_expectations": 0,
    "success_percent": null
  },
  "success": true,
  "meta": {
    "great_expectations_version": "0.14.13",
    "expectation_suite_name": "default",
    "run_id": {
      "run_time": "2022-08-13T12:31:49.160753+00:00",
      "run_name": null
    },
    "batch_kwargs": {
      "ge_batch_id": "e3236721-1b03-11ed-b9f1-83b88549211a"
    },
    "batch_markers": {},
    "batch_parameters": {},
    "validation_time": "20220813T123149.160715Z",
    "expectation_suite_meta": {
      "great_expectations_version": "0.14.13"
    }
  },
  "results": [],
  "evaluation_parameters": {}
}

In [16]:
ds.get_profile(profiler=user_features_profiler_fail2)

<GEProfile with expectations: [
  {
    "kwargs": {
      "column": "driver_id"
    },
    "meta": {},
    "expectation_type": "expect_column_to_exist"
  }
]>

In [11]:
training_data_job.to_df(validation_reference=validation_reference_fail)

/home/francisco/github/feast/sdk/python/feast/infra/offline_stores/offline_store.py:90: RuntimeWarning: Dataset validation is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


,driver_id,label_driver_reported_satisfaction,event_timestamp,conv_rate,acc_rate,avg_daily_trips
0,1002,5,2022-08-13 05:38:57.842689+00:00,0.534422,0.761452,841
1,1001,1,2022-08-13 06:03:57.842684+00:00,0.922234,0.012095,798
2,1003,3,2022-08-13 05:01:57.842691+00:00,0.733360,0.287259,925


In [16]:
from datetime import datetime
!feast materialize-incremental {datetime.now().isoformat()}

Materializing 1 feature views to 2022-08-12 23:34:21-06:00 into the sqlite online store.

driver_hourly_stats from 2022-08-12 11:34:23-06:00 to 2022-08-12 23:34:21-06:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 198.98it/s]


In [17]:
print("--- Data directory ---")
!ls data

import sqlite3
import pandas as pd
con = sqlite3.connect("data/online_store.db")
print("\n--- Schema of online store ---")
print(
    pd.read_sql_query("SELECT * FROM dqm_repo_driver_hourly_stats limit 10;", con)
)
con.close()

--- Data directory ---
driver_stats.parquet		online_store.db
driver_stats_reference.parquet	registry.db

--- Schema of online store ---
                                          entity_key     feature_name  \
0  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
1  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
2  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
3  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
4  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
5  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
6  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
7  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
8  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
9  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   

               value             event_ts                  c

In [18]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()

pprint(feature_vector)


{'acc_rate': [0.4692337214946747, 0.5439016222953796],
 'avg_daily_trips': [529, 114],
 'conv_rate': [0.8161515593528748, 0.20459125936031342],
 'driver_id': [1004, 1005]}


In [19]:
from feast import FeatureStore
feature_store = FeatureStore('.')  # Initialize the feature store

feature_service = feature_store.get_feature_service("driver_activity")
feature_vector = feature_store.get_online_features(
    features=feature_service,
    entity_rows=[
        # {join_key: entity_value}
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()
pprint(feature_vector)

{'acc_rate': [0.4692337214946747, 0.5439016222953796],
 'avg_daily_trips': [529, 114],
 'conv_rate': [0.8161515593528748, 0.20459125936031342],
 'driver_id': [1004, 1005]}


In [20]:
from feast.saved_dataset import ValidationReference

ref = ValidationReference(
    name='user_features_training_ref',
    dataset_name="reference_dataset",
    profiler=user_features_profiler,
)

In [21]:
store.apply(ref)

In [23]:
import pandas as pd
insert_df = pd.DataFrame({
    "driver_id": [1003],
    "conv_rate": [-1],
    "acc_rate": [2],
    "avg_daily_trips": [1500],
    "event_timestamp": [datetime.now()],
    "created": [datetime.now()],
})

store.write_to_online_store("driver_hourly_stats", insert_df)

In [ ]:
end_ts = datetime.now()
start_ts = end_ts - timedelta(minutes=10)

! feast validate --feature-service driver_activity --reference user_features_training_ref {start_ts.isoformat()} {end_ts.isoformat()}